In [ ]:
%%capture

# Upgrade pip
!pip install --upgrade pip
# Connectivity
!pip install psycopg2-binary  # PostgreSQL adapter
# !pip install snowflake-connector-python  # Snowflake connector
!pip install snowflake-connector-python==3.15.0 # Snowflake connector Older Version
!pip install snowflake-sqlalchemy  # Snowflake SQLAlchemy connector
!pip install warnings # Warnings management
# !pip install pyarrow # Serialization
!pip install keyring==23.11.0 # Key management
!pip install sqlalchemy==1.4.46 # SQLAlchemy
!pip install requests # HTTP requests
!pip install boto3 # AWS SDK
# !pip install slackclient # Slack API
!pip install oauth2client # Google Sheets API
!pip install gspread==5.9.0 # Google Sheets API
!pip install gspread_dataframe # Google Sheets API
!pip install google.cloud # Google Cloud
# Data manipulation and analysis
!pip install polars
!pip install pandas==2.2.1
!pip install numpy
# !pip install fastparquet
!pip install openpyxl # Excel file handling
!pip install xlsxwriter # Excel file handling
# Linear programming
!pip install pulp
# Date and time handling
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
# Progress bar
!pip install tqdm
# Database data types
!pip install db-dtypes
# Geospatial data handling
# !pip install geopandas
# !pip install shapely
# !pip install fiona
# !pip install haversine
# Plotting

# Modeling
!pip install statsmodels
!pip install scikit-learn

!pip install import-ipynb

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import calendar
import json
from datetime import date, timedelta
from oauth2client.service_account import ServiceAccountCredentials
import setup_environment_2
import importlib
import import_ipynb
import warnings
import demand_sku_cntrb
import time
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()
import gspread

/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json
/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


In [6]:
def query_snowflake(query, columns=[]):
    import os
    import snowflake.connector
    import numpy as np
    import pandas as pd
    con = snowflake.connector.connect(
        user =  os.environ["SNOWFLAKE_USERNAME"],
        account= os.environ["SNOWFLAKE_ACCOUNT"],
        password= os.environ["SNOWFLAKE_PASSWORD"],
        database =os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        if len(columns) == 0:
            out = pd.DataFrame(np.array(cur.fetchall()))
        else:
            out = pd.DataFrame(np.array(cur.fetchall()),columns=columns)
        return out
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        con.close()

In [7]:
query = '''
SHOW PARAMETERS LIKE 'TIMEZONE'
'''
x  = query_snowflake(query)
zone_to_use = x[1].values[0]

In [8]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
import requests
    
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])

        
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
username = pricing_api_secret["egypt_username"]
password = pricing_api_secret["egypt_password"]
secret = pricing_api_secret["egypt_secret"]

# get access token
def get_access_token(url, client_id, client_secret):
    """
    get_access_token function takes three parameters and returns a session token
    to connect to MaxAB APIs

    :param url: production MaxAB token URL
    :param client_id: client ID
    :param client_secret: client sercret
    :return: session token
    """
    response = requests.post(
        url,
        data={"grant_type": "password",
              "username": username,
              "password": password},
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def post_prices(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/pricing".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

def post_cart_rules(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/cart-rules".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

In [9]:
query = f'''
with vec_cost_type as(

select region,CATEGORY as vehicle_name,COST_PER_VEHICLE_RUN as vehicle_cost
from (
WITH fms AS (WITH reasons AS (SELECT DISTINCT
fleet_cost_mapping.reason_id,
logs_reason.reason,
CASE WHEN regions.name_en IN ('Delta West','Delta East') THEN 'Delta' ELSE regions.name_en END AS region,
cost

FROM retool.fleet_cost_mapping

INNER JOIN regions ON fleet_cost_mapping.region_id = regions.id
LEFT JOIN retool.logs_reason ON fleet_cost_mapping.reason_id = logs_reason.id),

vehicles AS (WITH history AS (
    SELECT DISTINCT
        SUPPLIERS_VEHICLES_history.supplier_id,
        supplier.name_en AS supplier_en,
        supplier.name_ar AS supplier_ar,
        supplier.tax_number,
        supplier.tax_name,
        SUPPLIERS_VEHICLES_history.VEHICLE_PLATE_NUMBER,
        SUPPLIERS_VEHICLES_history.monthly_cost,
        SUPPLIERS_VEHICLES_history.created_at AS created_timestamp,
        SUPPLIERS_VEHICLES_history.created_at::DATE AS created_at
    FROM retool.SUPPLIERS_VEHICLES_history
    LEFT JOIN retool.supplier ON SUPPLIERS_VEHICLES_history.supplier_id = supplier.id
),
history_with_next_update AS (
    SELECT
        *,
        COALESCE(LEAD(created_at) OVER (PARTITION BY VEHICLE_PLATE_NUMBER ORDER BY created_timestamp), CURRENT_DATE + 1) AS next_update 
    FROM history
)
SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY VEHICLE_PLATE_NUMBER ORDER BY created_at) AS ranks
FROM history_with_next_update
WHERE created_at <> next_update OR next_update IS NULL)

SELECT
fleet_tracking.delivery_date::VARCHAR AS date,
lh_warehouses.region,
warehouses.name AS warehouse,
COALESCE(vehicles.supplier_en, V2.supplier_en) AS supplier_en,
COALESCE(vehicles.supplier_ar, V2.supplier_ar) AS supplier_ar,
COALESCE(vehicles.tax_number, V2.tax_number) AS tax_number,
COALESCE(vehicles.tax_name, V2.tax_name) AS tax_name,
fleet_tracking.vehicle_plate_number AS plate_number,
fleet_tracking.run_sheet_id,
vehicle_types.name_en AS category,
fleet_tracking_liability.liability,
fleet_logs.logs,
--fleet_tracking.cost,
CASE WHEN fleet_tracking.logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END AS vehicle_cost,
--CASE WHEN fleet_tracking.logs_id IN (1,2) THEN COALESCE(history.monthly_cost,0)/26.0 ELSE NULL END AS vehicle_cost,
fleet_tracking.ON_DEMAND_COST,
extra_reasons.reason AS extra_reason,
extra_reasons.cost AS extra_cost,

case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.extra)[0]:amount::INT end  as new_extra_amount_1,
case when fleet_tracking.approved = 'true' then extra_reasons_1.reason::VARCHAR end as extra_reasons_1 ,
case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.extra)[1]:amount::INT end as new_extra_amount_2,
case when fleet_tracking.approved = 'true' then extra_reasons_2.reason::VARCHAR  end as extra_reasons_2,
case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.extra)[2]:amount::INT end  as new_extra_amount_3,
case when fleet_tracking.approved = 'true' then extra_reasons_3.reason::VARCHAR  end as extra_reasons_3,
COALESCE(new_extra_amount_1,0) + COALESCE(new_extra_amount_2,0) + COALESCE(new_extra_amount_3,0) + COALESCE(extra_reasonS.cost,0) as total_extra_cost , 





deduction_reasons.reason AS deduction_reason,
deduction_reasons.cost AS deduction_cost,

case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.deduction)[0]:amount::INT end  as new_deduction_amount_1,
case when fleet_tracking.approved = 'true' then deduction_reason_1.reason::VARCHAR  end as deduction_reason_1,
case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.deduction)[1]:amount::INT  end as new_deduction_amount_2,
case when fleet_tracking.approved = 'true' then deduction_reason_2.reason::VARCHAR end as deduction_reason_2,
case when fleet_tracking.approved = 'true' then parse_json(fleet_tracking.deduction)[2]:amount::INT end as new_deduction_amount_3,
case when fleet_tracking.approved = 'true' then deduction_reason_3.reason::VARCHAR end as deduction_reason_3,

COALESCE(new_deduction_amount_1,0) + COALESCE(new_deduction_amount_2,0) + COALESCE(new_deduction_amount_3,0)  as total_deduction_cost , 




waiting_reasons.reason AS waiting_reason,

CASE WHEN fleet_tracking.logs_id IN (4,5) AND fleet_tracking.waiting_reason_id = 13 THEN vehicles.monthly_cost/26.0 END AS waiting_cost,
fleet_tracking.karta_cost,
fleet_tracking.adjustment_cost,

(COALESCE(CASE WHEN fleet_tracking.logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END, 0) + 

 COALESCE(fleet_tracking.ON_DEMAND_COST,0)+ 
 
COALESCE(extra_reasons.cost, 0) + 

COALESCE((CASE WHEN fleet_tracking.logs_id IN (4,5) AND fleet_tracking.waiting_reason_id = 13 THEN vehicles.monthly_cost/26.0 END), 0)
+
COALESCE(deduction_reasons.cost, 0)) 
+ 
COALESCE(new_extra_amount_1,0) + COALESCE(new_extra_amount_2,0) + COALESCE(new_extra_amount_3,0)
-
(COALESCE(new_deduction_amount_1,0) + COALESCE(new_deduction_amount_3,0) + COALESCE(new_deduction_amount_3,0))
- 
COALESCE(fleet_tracking.adjustment_cost,0) AS final_cost,
fleet_force_delay.REVISED_DELIVERY_DATE::VARCHAR AS new_delivery_date,
updated_logs.logs AS updated_logs,
CASE WHEN fleet_force_delay.updated_logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END AS updated_vehicle_cost,
updated_extra_reasons.reason AS updated_extra_reason,

case when fleet_force_delay.approved = 'true' then parse_json(fleet_force_delay.extra)[0]:amount::INT end  as updated_new_extra_amount_1,
case when fleet_force_delay.approved = 'true' then extra_reasons_1.reason::VARCHAR end as updated_extra_reasons_1 ,
case when fleet_force_delay.approved = 'true' then parse_json(fleet_force_delay.extra)[1]:amount::INT end as updated_new_extra_amount_2,
case when fleet_force_delay.approved = 'true' then extra_reasons_2.reason::VARCHAR  end as updated_extra_reasons_2,
case when fleet_force_delay.approved = 'true' then parse_json(fleet_force_delay.extra)[2]:amount::INT end  as updated_new_extra_amount_3,
case when fleet_force_delay.approved = 'true' then extra_reasons_3.reason::VARCHAR  end as updated_extra_reasons_3,






updated_extra_reasons.cost AS updated_extra_cost,
updated_deduction_reasons.reason AS updated_deduction_reason,
updated_deduction_reasons.cost AS updated_deduction_cost,
fleet_force_delay.karta_cost AS updated_karta_cost,
fleet_force_delay.ON_DEMAND_COST AS UPDATED_ON_DEMAND_COST,
COALESCE(CASE WHEN fleet_force_delay.updated_logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END, 0) + 
COALESCE(updated_extra_reasons.cost, 0) + COALESCE(UPDATED_ON_DEMAND_COST,0) + 
COALESCE(updated_deduction_reasons.cost, 0) + (COALESCE(updated_new_extra_amount_1,0) + COALESCE(updated_new_extra_amount_2,0) + COALESCE(updated_new_extra_amount_3,0) )  AS force_delay_final_cost,
((COALESCE(CASE WHEN fleet_tracking.logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END, 0) + 
COALESCE(extra_reasons.cost, 0) + 
COALESCE(UPDATED_ON_DEMAND_COST,0) + 
COALESCE((CASE WHEN fleet_tracking.logs_id IN (4,5) AND fleet_tracking.waiting_reason_id = 13 THEN vehicles.monthly_cost/26.0 END), 0) +
COALESCE(deduction_reasons.cost, 0)) - 
COALESCE(fleet_tracking.adjustment_cost,0)) + 
(COALESCE(CASE WHEN fleet_force_delay.updated_logs_id IN (1,2) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26.0 ELSE NULL END, 0) + 
COALESCE(updated_extra_reasons.cost, 0) + 
COALESCE(fleet_tracking.ON_DEMAND_COST,0) +
COALESCE(updated_deduction_reasons.cost, 0))
+
total_extra_cost + 
(COALESCE(updated_new_extra_amount_1,0) + COALESCE(updated_new_extra_amount_2,0) + COALESCE(updated_new_extra_amount_3,0) )
- (COALESCE(new_deduction_amount_1,0) + COALESCE(new_deduction_amount_2,0) + COALESCE(new_deduction_amount_3,0) )
AS total_amount

FROM retool.fleet_tracking

LEFT JOIN vehicles ON REPLACE(fleet_tracking.VEHICLE_PLATE_NUMBER, ' ', '') = REPLACE(vehicles.VEHICLE_PLATE_NUMBER, ' ', '') 
AND fleet_tracking.delivery_date::DATE >= vehicles.created_at AND fleet_tracking.delivery_date::DATE < vehicles.next_update

LEFT JOIN vehicles V2 ON REPLACE(fleet_tracking.VEHICLE_PLATE_NUMBER, ' ', '') = REPLACE(v2.VEHICLE_PLATE_NUMBER, ' ', '') 
AND V2.ranks = 1

LEFT JOIN materialized_views.lh_warehouses ON fleet_tracking.warehouse_id = lh_warehouses.id
LEFT JOIN retool.suppliers_vehicles ON REPLACE(fleet_tracking.vehicle_plate_number, ' ', '') = suppliers_vehicles.vehicle_plate_number
LEFT JOIN retool.supplier ON suppliers_vehicles.supplier_id = supplier.id
LEFT JOIN warehouses ON fleet_tracking.warehouse_id = warehouses.id
LEFT JOIN drivers ON fleet_tracking.driver_id = drivers.id
LEFT JOIN vehicle_types ON fleet_tracking.vehicle_id = vehicle_types.id
LEFT JOIN retool.fleet_tracking_liability ON fleet_tracking.liability_id = fleet_tracking_liability.id

LEFT JOIN retool.fleet_logs ON fleet_tracking.logs_id = fleet_logs.id
LEFT JOIN reasons extra_reasons ON fleet_tracking.extra_reason_id = extra_reasons.reason_id AND extra_reasons.region = lh_warehouses.region
LEFT JOIN reasons deduction_reasons ON fleet_tracking.deduction_reason_id = deduction_reasons.reason_id AND deduction_reasons.region = lh_warehouses.region
LEFT JOIN retool.logs_reason waiting_reasons ON fleet_tracking.waiting_reason_id = waiting_reasons.id

LEFT JOIN retool.fleet_force_delay ON fleet_tracking.run_sheet_id = fleet_force_delay.run_sheet_id
LEFT JOIN retool.fleet_logs updated_logs ON fleet_force_delay.updated_logs_id = updated_logs.id
LEFT JOIN reasons updated_deduction_reasons ON fleet_force_delay.deduction_reason_id = updated_deduction_reasons.reason_id AND updated_deduction_reasons.region = lh_warehouses.region
LEFT JOIN reasons updated_extra_reasons ON fleet_force_delay.extra_reason_id = updated_extra_reasons.reason_id AND updated_extra_reasons.region = lh_warehouses.region


LEFT JOIN retool.logs_reason as extra_reasons_1
ON extra_reasons_1.ID::INT  = parse_json(fleet_tracking.extra)[0]:extra_reason_id::INT

LEFT JOIN retool.logs_reason as deduction_reason_1
ON deduction_reason_1.ID::INT  = parse_json(fleet_tracking.deduction)[0]:deduction_reason_id::INT

LEFT JOIN retool.logs_reason as extra_reasons_2
ON extra_reasons_2.ID::INT  = parse_json(fleet_tracking.extra)[1]:extra_reason_id::INT

LEFT JOIN retool.logs_reason as deduction_reason_2
ON deduction_reason_2.ID::INT  = parse_json(fleet_tracking.deduction)[1]:deduction_reason_id::INT

LEFT JOIN retool.logs_reason as extra_reasons_3
ON extra_reasons_3.ID::INT  = parse_json(fleet_tracking.extra)[2]:extra_reason_id::INT

LEFT JOIN retool.logs_reason as deduction_reason_3
ON deduction_reason_3.ID::INT  = parse_json(fleet_tracking.deduction)[2]:deduction_reason_id::INT



WHERE fleet_tracking.delivery_date::DATE BETWEEN date_trunc('month',current_date - interval '1 month' ) and date_trunc('month',current_date)-1 
-- and fleet_tracking.run_sheet_id = 1374549


UNION

SELECT DISTINCT
unutilized_vehicle.date::VARCHAR AS date,
lh_warehouses.region,
warehouses.name AS warehouse,
COALESCE(vehicles.supplier_en, V2.supplier_en) AS supplier_en,
COALESCE(vehicles.supplier_ar, V2.supplier_ar) AS supplier_ar,
COALESCE(vehicles.tax_number, V2.tax_number) AS tax_number,
COALESCE(vehicles.tax_name, V2.tax_name) AS tax_name,
unutilized_vehicle.vehicle_plate_number AS plate_number,
0 AS run_sheet_id,
vehicle_types.name_en AS category,
'' AS liability,
'Unutilized' AS logs,
0 AS vehicle_cost,
0 as ON_DEMAND_COST,
'' AS extra_reason,
0 AS extra_cost,

0 as new_extra_amount_1,
'' AS  extra_reasons_1,
0 as new_extra_amount_2,
'' as extra_reasons_2,
0 as new_extra_amount_3,
'' as extra_reasons_3 , 
0  AS total_extra_cost ,


'' AS deduction_reason,
0 AS deduction_cost,


0 as new_deduction_amount_1 ,
'' as deduction_reason_1 ,
0 as new_deduction_amount_2 ,
'' as deduction_reason_2 , 
0 as new_deduction_amount_3 ,
'' as deduction_reason_3,
0 as total_deduction_cost,




'' AS waiting_reason,
(CASE WHEN DATE BETWEEN '2024-04-01' AND '2024-04-16' THEN      
(
CASE WHEN region = 'Greater Cairo' THEN 
          CASE
               WHEN (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
               WHEN (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN 150 END
      ELSE (CASE WHEN (unutilized_vehicle.arrived = true) THEN (COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26)* 0.5 END) END )
      ELSE 
      (
      CASE WHEN DATE < '2024-04-01'
      THEN 
      (CASE WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
      WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN 150 END)
      
      ELSE 
            ROUND(CASE
          WHEN region = 'Greater Cairo' THEN 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * unutilized_vehicle.reason_id 
              END
          ELSE 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * 0.5
              END
      END) END ) END ) AS waiting_cost,
0 AS karta_cost,
unutilized_vehicle.adjustment_cost,

 (CASE WHEN DATE BETWEEN '2024-04-01' AND '2024-04-16' THEN      
(
CASE WHEN region = 'Greater Cairo' THEN 
          CASE
               WHEN (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
               WHEN (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN 150 END
      ELSE (CASE WHEN (unutilized_vehicle.arrived = true) THEN (COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26)* 0.5 END) END )
      ELSE 
      (
      CASE WHEN DATE < '2024-04-01'
      THEN 
      (CASE WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
      WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN 150 END)
      
      ELSE 
            ROUND(CASE
          WHEN region = 'Greater Cairo' THEN 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * unutilized_vehicle.reason_id 
              END
          ELSE 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * 0.5
              END
      END) END ) END )
      
      
-- (CASE WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
--       WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN 150 END)
      
      - 
COALESCE(unutilized_vehicle.adjustment_cost,0) AS final_cost,
 ''  AS new_delivery_date,
'' AS updated_logs,
0 AS updated_vehicle_cost,
'' AS updated_extra_reason,

0 as updated_new_extra_amount_1,
'' AS  updated_extra_reasons_1,
0 as updated_new_extra_amount_2,
'' as updated_extra_reasons_2,
0 as updated_new_extra_amount_3,
'' as updated_extra_reasons_3 , 

0 AS updated_extra_cost,




'' AS updated_deduction_reason,
0 AS updated_deduction_cost,
0 AS updated_karta_cost,
0 AS UPDATED_ON_DEMAND_COST,
0 AS force_delay_final_cost,

 -- unutilized_vehicle_cost
-- (CASE WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
--       WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN 150 END) 
 
(CASE WHEN DATE BETWEEN '2024-04-01' AND  '2024-04-16' THEN      
(
CASE WHEN region = 'Greater Cairo' THEN 
          CASE
               WHEN (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
               WHEN (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true) THEN 150 END
      ELSE (CASE WHEN (unutilized_vehicle.arrived = true) THEN (COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26)* 0.5 END) END )
      ELSE 
      (
      CASE WHEN DATE < '2024-04-01'
      THEN 
      (CASE WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN COALESCE(vehicles.monthly_cost, v2.monthly_cost)/26
      WHEN (unutilized_vehicle.reason_id IS NOT NULL) AND (suppliers_vehicles.vehicle_id NOT IN (67,68,100)) AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL) THEN 150 END)
      
      ELSE 
            ROUND(CASE
          WHEN region = 'Greater Cairo' THEN 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * unutilized_vehicle.reason_id 
              END
          ELSE 
              CASE
                  WHEN unutilized_vehicle.arrived = true THEN (vehicles.monthly_cost / 26) * 0.5
              END
      END) END ) END ) - COALESCE(unutilized_vehicle.adjustment_cost,0) +
(COALESCE(new_extra_amount_1,0) + COALESCE(new_extra_amount_2,0) + COALESCE(new_extra_amount_3,0) ) 
+ 
(COALESCE(updated_new_extra_amount_1,0) + COALESCE(updated_new_extra_amount_2,0) + COALESCE(updated_new_extra_amount_3,0) ) 
-
(COALESCE(new_deduction_amount_1,0) + COALESCE(new_deduction_amount_2,0) + COALESCE(new_deduction_amount_3,0) )
AS total_amount

FROM retool.unutilized_vehicle

LEFT JOIN (WITH history AS (
    SELECT DISTINCT
        SUPPLIERS_VEHICLES_history.supplier_id,
        supplier.name_en AS supplier_en,
        supplier.name_ar AS supplier_ar,
        supplier.tax_number,
        supplier.tax_name,
        SUPPLIERS_VEHICLES_history.VEHICLE_PLATE_NUMBER,
        SUPPLIERS_VEHICLES_history.monthly_cost,
        SUPPLIERS_VEHICLES_history.created_at AS created_timestamp,
        SUPPLIERS_VEHICLES_history.created_at::DATE AS created_at
    FROM retool.SUPPLIERS_VEHICLES_history
    LEFT JOIN retool.supplier ON SUPPLIERS_VEHICLES_history.supplier_id = supplier.id
),
history_with_next_update AS (
    SELECT
        *,
        COALESCE(LEAD(created_at) OVER (PARTITION BY VEHICLE_PLATE_NUMBER ORDER BY created_timestamp), CURRENT_DATE + 1) AS next_update 
    FROM history
)
SELECT
    *
FROM history_with_next_update
WHERE created_at <> next_update OR next_update IS NULL) vehicles ON REPLACE(unutilized_vehicle.VEHICLE_PLATE_NUMBER, ' ', '') = REPLACE(vehicles.VEHICLE_PLATE_NUMBER, ' ', '') 
AND unutilized_vehicle.date::DATE >= vehicles.created_at AND unutilized_vehicle.date < vehicles.next_update

LEFT JOIN (WITH history AS (
    SELECT DISTINCT
        SUPPLIERS_VEHICLES_history.supplier_id,
        supplier.name_en AS supplier_en,
        supplier.name_ar AS supplier_ar,
        supplier.tax_number,
        supplier.tax_name,
        SUPPLIERS_VEHICLES_history.VEHICLE_PLATE_NUMBER,
        SUPPLIERS_VEHICLES_history.monthly_cost,
        SUPPLIERS_VEHICLES_history.created_at AS created_timestamp,
        SUPPLIERS_VEHICLES_history.created_at::DATE AS created_at
    FROM retool.SUPPLIERS_VEHICLES_history
    LEFT JOIN retool.supplier ON SUPPLIERS_VEHICLES_history.supplier_id = supplier.id
),
history_with_next_update AS (
    SELECT
        *,
        COALESCE(LEAD(created_at) OVER (PARTITION BY VEHICLE_PLATE_NUMBER ORDER BY created_timestamp), CURRENT_DATE + 1) AS next_update 
    FROM history
)
SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY VEHICLE_PLATE_NUMBER ORDER BY created_at) AS ranks
FROM history_with_next_update
WHERE created_at <> next_update OR next_update IS NULL) v2 ON REPLACE(unutilized_vehicle.VEHICLE_PLATE_NUMBER, ' ', '') = REPLACE(v2.VEHICLE_PLATE_NUMBER, ' ', '') 
AND v2.ranks = 1

LEFT JOIN warehouses ON unutilized_vehicle.warehouse_id = warehouses.id
LEFT JOIN retool.suppliers_vehicles ON REPLACE(unutilized_vehicle.vehicle_plate_number, ' ', '') = suppliers_vehicles.vehicle_plate_number
LEFT JOIN retool.supplier ON suppliers_vehicles.supplier_id = supplier.id
LEFT JOIN retool.logs_reason ON unutilized_vehicle.reason_id = logs_reason.id
LEFT JOIN vehicle_types ON suppliers_vehicles.vehicle_id = vehicle_types.id
LEFT JOIN materialized_views.lh_warehouses ON unutilized_vehicle.warehouse_id = lh_warehouses.id

WHERE unutilized_vehicle.date::DATE  BETWEEN date_trunc('month',current_date - interval '1 month' ) and date_trunc('month',current_date)-1 
AND (unutilized_vehicle.arrived = true OR unutilized_vehicle.arrived IS NULL)

)


SELECT 
date_trunc('month',date::date) month,
 region,
category,
count(distinct run_sheet_id) vehicles,
sum(total_amount) total_amountt,
total_amountt/vehicles cost_per_vehicle_run 
FROM fms
where logs in ('first')
 GROUP BY all
order by 1,2,3
)

),

vec as (
select vct.region , vt.id as vehicle_id,name_en as vehicle_name,vc.weight as vehicle_weight,vc.cbm as vehicle_cbm,vehicle_cost+300 as vehicle_cost
from vec_cost_type vct
join VEHICLE_TYPES  vt on vt.name_en = vct.vehicle_name
join  RETOOL.VEHICLE_CAPACITIES vc on vc.vehicle_id = vt.id
where vehicle_id = 100
),
products_dim as (
select product_id,is_basic_unit,packing_unit_id,(long * width * height)/1000000.00 AS cbm,weight/1000.00 AS weight,
from (
SELECT DISTINCT
product_id, packing_unit_id, long, width, height, weight,is_basic_unit
FROM packing_unit_products
join products on products.id = packing_unit_products.product_id 
where  is_basic_unit = 1
and products.ACTIVATION = 'true'
)
),
vehicle_cost as (
select * ,vehicle_cost/quantity as vehicle_cost_per_unit 
from (
select region,product_id,packing_unit_id,vehicle_name,vehicle_id,cbm,weight,least(cbm_qty,weight_qty) as quantity ,vehicle_cost as vehicle_cost
from(
select *, floor((vehicle_cbm*0.95)/cbm) as cbm_qty , floor((vehicle_weight*0.9)/weight) as weight_qty  
from  products_dim , vec
)
)
),
wh_cost as (
select product_id,region,sum(wh_cost)/sum(nmv_ex_vat) as wh_cost_perc , sum(lm_cost)/sum(nmv_ex_vat) as LM_cost_perc
from finance.sku_costs c 
where month >= date_trunc('month',current_date - interval '10 months')
and WH_COST > 0 
and nmv_ex_vat > 0 
and lm_cost > 0 
group by all 
),
mapping as (
SELECT *
             FROM   (values
                            ('Greater Cairo','Cairo',700),
                            ('Greater Cairo','Giza', 701),
                            ('Delta','Delta West',703),
                            ('Delta','Delta East', 704),
                            ('Upper Egypt','Upper Egypt', 1124),
                            ('Upper Egypt','Upper Egypt', 1126),
                            ('Upper Egypt','Upper Egypt',1123),
                            ('Upper Egypt','Upper Egypt',1125),
                            ('Alexandria','Alexandria',702))
							
                    x(main_region,region,cohort_id)
					),

skus_prices as (
with local_prices as (
SELECT  case when cpu.cohort_id in (700,695) then 'Cairo'
             when cpu.cohort_id in (701) then 'Giza'
             when cpu.cohort_id in (704,698) then 'Delta East'
             when cpu.cohort_id in (703,697) then 'Delta West'
             when cpu.cohort_id in (696,1123,1124,1125,1126) then 'Upper Egypt'
             when cpu.cohort_id in (702,699) then 'Alexandria'
        end as region,
		cohort_id,
        pu.product_id,
		pu.packing_unit_id as packing_unit_id,
		pu.basic_unit_count,
        avg(cpu.price) as price
FROM    cohort_product_packing_units cpu
join    PACKING_UNIT_PRODUCTS pu on pu.id = cpu.product_packing_unit_id
WHERE   cpu.cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
    and cpu.created_at::date<>'2023-07-31'
    and cpu.is_customized = true
	group by all 
),
live_prices as (
select region,cohort_id,product_id,pu_id as packing_unit_id,buc as basic_unit_count,NEW_PRICE as price
from materialized_views.DBDP_PRICES
where created_at = current_date
and DATE_PART('hour',CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND SPLIT_PART(time_slot, '-', 2)::int
and cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
),
prices as (
select *
from (
    SELECT *, 1 AS priority FROM live_prices
    UNION ALL
    SELECT *, 2 AS priority FROM local_prices
)
QUALIFY ROW_NUMBER() OVER (PARTITION BY region,cohort_id,product_id,packing_unit_id ORDER BY priority) = 1
)
select region,cohort_id,product_id,price 
from prices 
where basic_unit_count = 1
AND (
        (product_id = 1309 AND packing_unit_id = 2)
        OR (product_id <> 1309)
      )
)
select *,(wac_p + vehicle_cost_per_unit) as wac_ws,
(wac_p + (wac_p*lm_cost_perc)) as wac_norm,
case when price <> 0 then (price-wac_ws)/price else 0 end as cm3_whole_sale,
case when price <> 0 then (price-wac_norm)/price else 0 end as cm3_norm
from (
select m.cohort_id,vc.product_id,CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,b.name_ar as brand,cat.name_ar as cat,m.region,packing_unit_id,vehicle_name,vehicle_id,vc.quantity,vc.vehicle_cost,vehicle_cost_per_unit,coalesce(wh_cost_perc,0) as wh_cost_perc,coalesce(LM_cost_perc,0) as LM_cost_perc,max(wac1) as wac1,max(wac_p) as wac_p,p.price
from vehicle_cost vc  
left join wh_cost wc on vc.product_id = wc.product_id and wc.region = vc.region
join mapping m on m.main_region = vc.region
join finance.all_cogs c on c.product_id = vc.product_id and CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp()) between c.from_date and c.to_date
join skus_prices p on p.product_id = vc.product_id and p.cohort_id = m.cohort_id 
join products  on products.id = vc.product_id 
join categories cat on cat.id = products.category_id
join brands b on b.id = products.brand_id
JOIN product_units ON product_units.id = products.unit_id 
where wac1 > 0 and wac_p > 0 
group by all
)x

where region = 'Cairo'
and price > 0
'''
whole_sale = query_snowflake(query, columns = ['COHORT_ID','PRODUCT_ID','SKU','BRAND','CAT','REGION','PACKING_UNIT_ID','VEHICLE_NAME','VEHICLE_ID','QUANTITY','VEHICLE_COST','VEHICLE_COST_PER_UNIT','WH_COST_PERC','LM_COST_PERC','WAC1','WAC_P','PRICE','WAC_WS','WAC_NORM','CM3_WHOLE_SALE','CM3_NORM'])
whole_sale.columns = whole_sale.columns.str.lower()
for col in whole_sale.columns:
    whole_sale[col] = pd.to_numeric(whole_sale[col], errors='ignore')    

In [10]:
query = '''
SELECT DISTINCT cat, brand, margin as target_bm
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
'''
brand_cat_target  = query_snowflake(query, columns = ['cat','brand','target_bm'])
brand_cat_target.target_bm=pd.to_numeric(brand_cat_target.target_bm)

query = '''
select cat,sum(target_bm *(target_nmv/cat_total)) as cat_target_margin
from (
select *,sum(target_nmv)over(partition by cat) as cat_total
from (
select cat,brand,avg(target_bm) as target_bm , sum(target_nmv) as target_nmv
from (
SELECT DISTINCT date,city as region,cat, brand, margin as target_bm,nmv as target_nmv
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
)
group by all
)
)
group by all 
'''
cat_target  = query_snowflake(query, columns = ['cat','cat_target_margin'])
cat_target.cat_target_margin=pd.to_numeric(cat_target.cat_target_margin)

In [11]:
query = '''
SELECT  DISTINCT
		cpc.cohort_id,  
		pso.product_id,
        sum(pso.total_price) as nmv,
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
join COHORT_PRICING_CHANGES cpc on cpc.id = pso.COHORT_PRICING_CHANGE_ID
WHERE so.created_at::date between date_trunc('month', current_date- interval '3 months') and  date_trunc('month',current_date)-1
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
'''
sales  = query_snowflake(query, columns = ['cohort_id','product_id','nmv'])
sales.columns = sales.columns.str.lower()
for col in sales.columns:
    sales[col] = pd.to_numeric(sales[col], errors='ignore')    

In [12]:
query = '''
select * 
from materialized_views.sku_commercial_groups
'''
groups  = setup_environment_2.dwh_pg_query(query, columns = ['product_id','group'])
groups.columns = groups.columns.str.lower()
for col in groups.columns:
    groups[col] = pd.to_numeric(groups[col], errors='ignore')      
    

In [13]:
query = '''

WITH whs as (SELECT *
             FROM   (values
                            ('Cairo', 'Mostorod', 1,700),
                            ('Giza', 'Barageel', 236,701),
                            ('Delta West', 'El-Mahala', 337,703),
                            ('Delta West', 'Tanta', 8,703),
                            ('Delta East', 'Mansoura FC', 339,704),
                            ('Delta East', 'Sharqya', 170,704),
                            ('Upper Egypt', 'Assiut FC', 501,1124),
                            ('Upper Egypt', 'Bani sweif', 401,1126),
                            ('Upper Egypt', 'Menya Samalot', 703,1123),
                            ('Upper Egypt', 'Sohag', 632,1125),
                            ('Alexandria', 'Khorshed Alex', 797,702),
							('Giza', 'Sakkarah', 962,701)
							
							)
                    x(region, wh, warehouse_id,cohort_id))
					
select cohort_id,product_id,sum(stocks) as stocks 
from (
		SELECT DISTINCT whs.region,
				cohort_id,	
                whs.wh,
                product_warehouse.product_id,
                (product_warehouse.available_stock)::integer as stocks,

        from whs
        JOIN product_warehouse ON product_warehouse.warehouse_id = whs.warehouse_id
        JOIN products on product_warehouse.product_id = products.id
        JOIN product_units ON products.unit_id = product_units.id

        where   product_warehouse.warehouse_id not in (6,9,10)
            AND product_warehouse.is_basic_unit = 1
			and product_warehouse.available_stock > 0 

)
group by all

'''
stocks  = query_snowflake(query, columns = ['cohort_id','product_id','stocks'])
stocks.columns = stocks.columns.str.lower()
for col in stocks.columns:
    stocks[col] = pd.to_numeric(stocks[col], errors='ignore')     

In [14]:
command_string = '''
select 
product_id,
PACKING_UNIT_id,
basic_unit_count,
from PACKING_UNIT_PRODUCTS
where PACKING_UNIT_PRODUCTS.deleted_at is null
order by product_id,basic_unit_count'''
pu = query_snowflake(command_string, columns = ['product_id','pu_id', 'buc'])
pu.product_id = pd.to_numeric(pu.product_id)
pu.pu_id = pd.to_numeric(pu.pu_id)
pu.buc = pd.to_numeric(pu.buc)

In [15]:
query ='''
select product_id,new_pp,forecasted_date
from materialized_views.DBDP_PRICE_UPS
where region = 'Cairo'
'''
price_ups  = query_snowflake(query, columns = ['product_id','new_pp','forcasted_date'])
price_ups.columns = price_ups.columns.str.lower()
for col in price_ups.columns:
    price_ups[col] = pd.to_numeric(price_ups[col], errors='ignore')        

In [16]:
def select_price(x):
    old_cm3 = x['cm3_norm']
    new_cm3 = x['cm3_whole_sale']
    target_margin = x['target_margin']
    wac_p = x['wac_p'] 
    wac_ws = x['wac_ws'] 
    wac_norm = x['wac_norm']
    price = x['price'] 
    new_pp = x['new_pp']
    stocks = x['stocks']
    if new_cm3< old_cm3 or new_cm3 < 0:
        if(price > wac_p):
            return price 
        else:
            return round((wac_p/(1-target_margin))*4)/4
    else:
        if old_cm3 < 0:
            old_cm3=0
        distance = (new_cm3-old_cm3)
        if pd.isna(new_pp):
            return round((wac_ws/(1-(new_cm3-(0.9*distance))))*4)/4
        elif (~pd.isna(new_pp)) and stocks > 0:
            return round((wac_ws/(1-(new_cm3-(0.5*distance))))*4)/4
        else:
            return price

In [17]:
wholesale_data  = whole_sale.merge(brand_cat_target,on =['cat','brand'],how='left')
wholesale_data  = wholesale_data.merge(cat_target,on =['cat'],how='left')
wholesale_data  = wholesale_data.merge(price_ups,on =['product_id'],how='left')
wholesale_data = wholesale_data.merge(stocks,on=['product_id','cohort_id'],how='left')
wholesale_data['stocks'] = wholesale_data['stocks'].fillna(0)
wholesale_data['target_margin'] = (wholesale_data['target_bm'].fillna(wholesale_data['cat_target_margin'])).fillna(0.04)
wholesale_data['selected_price'] =  wholesale_data.apply(select_price,axis=1)
wholesale_data['new_margin'] =  (wholesale_data['selected_price']-wholesale_data['wac_p'])/wholesale_data['selected_price']
wholesale_data['price_diff'] = (wholesale_data['selected_price']-wholesale_data['price'])/wholesale_data['price']
wholesale_data = wholesale_data.merge(sales,on=['product_id','cohort_id'],how='left')
wholesale_data['nmv'] = wholesale_data['nmv'].fillna(0)
wholesale_data['total_nmv'] = wholesale_data.groupby('cohort_id')['nmv'].transform(sum)
wholesale_data['cntrb'] = wholesale_data['nmv']/wholesale_data['total_nmv'] 
wholesale_data = wholesale_data.merge(groups,on=['product_id'],how='left')
wholesale_data['new_group_nmv'] = wholesale_data['nmv']
wholesale_data.loc[wholesale_data['stocks'] == 0 ,'new_group_nmv'] = wholesale_data['nmv']*0.1
wholesale_data['total_group_nmv'] =wholesale_data.groupby(['cohort_id','group'])['new_group_nmv'].transform(sum)
wholesale_data['price_cntrb'] = wholesale_data['selected_price'] * (wholesale_data['new_group_nmv']/wholesale_data['total_group_nmv'])
wholesale_data['final_group_price'] = wholesale_data.groupby(['cohort_id','group'])['price_cntrb'].transform(sum) 
wholesale_data['final_price'] = wholesale_data['final_group_price'].fillna(wholesale_data['selected_price'])
wholesale_data['final_price']=round(wholesale_data['final_price']*4)/4
wholesale_data.loc[wholesale_data['final_price']==0,'final_price'] = wholesale_data['selected_price']
wholesale_data.loc[wholesale_data['final_price']==0,'final_price'] = wholesale_data['price']

In [20]:
scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(setup_environment_2.get_secret("prod/maxab-sheets")), scope)
client = gspread.authorize(creds)
force_brands = client.open('Wholesales_exec').worksheet('brands')
force_cats = client.open('Wholesales_exec').worksheet('cats')
force_brands_df = pd.DataFrame(force_brands.get_all_records())
force_cats_df = pd.DataFrame(force_cats.get_all_records())
if force_brands_df.empty:
    forced_brand_list = []
else:    
    forced_brand_list = force_brands_df.brand.unique()
    
if force_cats_df.empty:
    forced_cat_list = []  
else:    
    forced_cat_list = force_cats_df.cat.unique()

In [21]:
wholesale_data.loc[wholesale_data['brand'].isin(forced_brand_list),'final_price'] = wholesale_data['price'] 
wholesale_data.loc[wholesale_data['cat'].isin(forced_cat_list),'final_price'] = wholesale_data['price'] 

In [23]:
final_data = wholesale_data[['cohort_id', 'product_id', 'sku', 'brand', 'cat','final_price']]
final_data=final_data.drop_duplicates()
final_data = final_data.merge(pu, on='product_id')
final_data['new_price'] = final_data['final_price'] * final_data['buc']
final_data['ind'] = 1
final_data['ind'] = final_data.groupby(['cohort_id', 'product_id']).ind.cumsum()
remove_min_pu = pd.read_csv('skus_to_remove_min.csv')
remove_min_pu['remove_min'] = 1
final_data = final_data.merge(remove_min_pu[['product_id','remove_min']], on='product_id', how='left')

In [24]:
cart_rules_data = final_data.copy()
cart_rules_data=cart_rules_data.merge(wholesale_data[['cohort_id','product_id','quantity']],on=['cohort_id','product_id'])
cart_rules_data['car_allowed_quantity'] = cart_rules_data['quantity']/(cart_rules_data['buc']*3)
cart_rules_data['half_allowed_quantity'] = 20000/(cart_rules_data['new_price'])
cart_rules_data['Cart_rules'] = np.ceil(np.minimum(np.minimum(cart_rules_data['half_allowed_quantity'],cart_rules_data['car_allowed_quantity']),100))
cart_rules_data.loc[cart_rules_data['brand'].isin(['بست','ريد بل','فيوري']),'Cart_rules']=7
cart_rules_data.loc[cart_rules_data['cat'].isin(['حاجه ساقعه']),'Cart_rules']=np.minimum(cart_rules_data['Cart_rules'],50)

In [25]:
to_upload = final_data[['product_id','sku','pu_id','new_price','cohort_id','ind','remove_min']]
to_upload=to_upload.drop_duplicates()
to_upload.dropna(subset=['new_price'], inplace=True)
to_upload = to_upload[to_upload.new_price > 0].drop_duplicates().reset_index(drop=True)

In [26]:
to_upload['cohort_id'] =1156
cart_rules_data['cohort_id'] =1156

In [27]:
cart_rules_data = cart_rules_data[['cohort_id','product_id','pu_id','Cart_rules']]

In [28]:
for cohort in to_upload.cohort_id.unique():
        upload = to_upload[to_upload['cohort_id']==cohort]
        out=upload[['product_id', 'sku', 'pu_id', 'new_price', 'ind', 'remove_min']].copy()
        out.columns = ['Product ID','Product Name','Packing Unit ID','Price','ind','remove_min']
        out['Visibility (YES/NO)'] = 'YES'
        out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
        out.drop(columns=['ind','remove_min'], inplace=True)
        out = out.drop_duplicates()
        out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
        out['Tags'] = None
        file_name_ = 'uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_')
        out.to_excel(file_name_,index = False,engine = 'xlsxwriter')
        time.sleep(5)
        ################### Loop to avoid file limit ######################
        # split file into chunks
        print('Spliting file into chunks...')
        if cohort == 61:
            chunks = [out[i:i + 2000] for i in range(0, len(out), 2000)]
        else:
            chunks = [out[i:i + 4000] for i in range(0, len(out), 4000)]
        print(f'len chunks = {len(chunks)}')
        fileslist = []
        for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
            fileslist.append(f'manual/output_{cohort}_chunk_{i + 1}.xlsx')
            output_file_path = f'manual/output_{cohort}_chunk_{i + 1}.xlsx'
            chunk.to_excel(output_file_path, index=False, engine='xlsxwriter')
        # Loop over chunks and upload
        print('Uploading...')
        for file in fileslist:
            chunk = file.split('chunk_')[1].split('.xls')[0]
            x = post_prices(cohort, file)
            # print(str(x.content))
            if ('"success":true' in str(x.content).lower()):
                print(f"Prices are upoladed successfuly cohort: {cohort}, chunk: {chunk}")
            else:
                print(f"ERROR cohort: {cohort}, chunk: {chunk}")
                print(x.content)
                final_status = False
                break

Spliting file into chunks...
len chunks = 3


100%|██████████| 3/3 [00:01<00:00,  2.87it/s]


Uploading...
Prices are upoladed successfuly cohort: 1156, chunk: 1
Prices are upoladed successfuly cohort: 1156, chunk: 2
Prices are upoladed successfuly cohort: 1156, chunk: 3


In [29]:
for cohort in cart_rules_data.cohort_id.unique():
    req_data = cart_rules_data[cart_rules_data['cohort_id']==cohort]
    if len(req_data) > 0 :
        req_data = req_data[['product_id','pu_id','Cart_rules']]
        req_data.columns = ['Product ID','Packing Unit ID','Cart Rules']
        req_data.to_excel(f'CartRules_{cohort}.xlsx', index=False, engine='xlsxwriter')
        time.sleep(5)
        x =  post_cart_rules(cohort,f'CartRules_{cohort}.xlsx')
        if x.ok:
            print(f"success_{cohort}")
        else:
            print(f"ERROR_{cohort}")
            print(x.content)
            break

success_1156
